## Prelims

### configs

In [1]:
GPU_INDEX=0
DATA_DIR="../../data/splits/stratified_general/"
FULL_CHV_PATH="../../data/chv.csv"
FEATURE_PATH="../../data/precomputed_features/"
SNOMED_PATH = "../../data/SnomedCT_201907/"

### imports

In [2]:
%load_ext autoreload
%autoreload 2

import torch
torch.manual_seed(2020) 
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '../../'))
from src.data import *
from src.loss import *
from src.models import *
from src.evaluation import *
from src.train import *

### load some dicts (to be used later)

In [3]:
# load SNOMED graph
from data.Snomed import Snomed
snomed = Snomed(SNOMED_PATH, taxonomy=False)
snomed.load_snomed()

In [4]:
# create surface_to_snomed_id dict
SF2ID = build_surface_to_snomed_id(snomed)

In [5]:
# create testset_row_index_to_ed_dict dict
ED_DICT = pkl.load(open(os.path.join(FEATURE_PATH,"term_ed_dic_stratified_general.pkl"),"rb"))

## Experiments

### BERT align

#### static term -> surface

In [6]:
# input: static term
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_surface_bert_ts100k_embeddings_all_names_mean_full_new.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"chv_plain_term_embeddings_BERTbr_ts100k.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path, 
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path,
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [27]:
model = fc_aligner(input_size=768, target_size=768).cuda(GPU_INDEX)
train_params = model.parameters()
optimizer = torch.optim.AdamW(train_params, lr=1e-4)
criterion = TripletLoss(margin=0.2, max_violation=True)

best_sd = train(model, train_params, optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0)

[epoch 0][dev ] Acc@1: 0.000 Acc@10: 0.000 Acc@100: 0.000 AccSum: 0.000
[epoch 1][dev ] Acc@1: 0.061 Acc@10: 0.193 Acc@100: 0.384 AccSum: 0.637
[epoch 1][test] Acc@1: 0.066 Acc@10: 0.188 Acc@100: 0.379 AccSum: 0.633
[best epoch: 1]
[epoch 2][dev ] Acc@1: 0.140 Acc@10: 0.347 Acc@100: 0.548 AccSum: 1.035
[epoch 2][test] Acc@1: 0.149 Acc@10: 0.353 Acc@100: 0.554 AccSum: 1.056
[best epoch: 2]
[epoch 3][dev ] Acc@1: 0.173 Acc@10: 0.393 Acc@100: 0.589 AccSum: 1.155
[epoch 3][test] Acc@1: 0.178 Acc@10: 0.398 Acc@100: 0.594 AccSum: 1.170
[best epoch: 3]
[epoch 4][dev ] Acc@1: 0.216 Acc@10: 0.437 Acc@100: 0.627 AccSum: 1.281
[epoch 4][test] Acc@1: 0.220 Acc@10: 0.446 Acc@100: 0.634 AccSum: 1.299
[best epoch: 4]
[epoch 5][dev ] Acc@1: 0.216 Acc@10: 0.442 Acc@100: 0.630 AccSum: 1.287
[epoch 5][test] Acc@1: 0.225 Acc@10: 0.452 Acc@100: 0.636 AccSum: 1.313
[best epoch: 5]
[epoch 6][dev ] Acc@1: 0.203 Acc@10: 0.432 Acc@100: 0.625 AccSum: 1.260
[epoch 7][dev ] Acc@1: 0.217 Acc@10: 0.450 Acc@100: 0.63

In [28]:
save_path = "bert_static_term_to_surface_state_dict.pkl"
torch.save(model.state_dict(), save_path)
model = fc_aligner(input_size=768, target_size=768).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

fc_aligner(
  (fc1): Linear(in_features=768, out_features=768, bias=True)
)

In [30]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128)
print ("MRR: %.3f" % mrr)

# gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=128)
# print ("MGRD: %.3f MGRD@10: %.3f" % gds)


MRR: 0.411


#### contextual_term -> surface

##### mutlilevel attention

In [6]:
# input: contextual_term
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_surface_bert_ts100k_embeddings_all_names_mean_full_last_layer_with_ST.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"chv_term_embeddings_BERTbr_ts100k_multilevel_all.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path,
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path, 
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [7]:
model = multilevel_attention(input_size=768, target_size=768, lin=True).cuda(GPU_INDEX)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = TripletLoss(margin=0.2, max_violation=True, device=GPU_INDEX)

best_sd = train(model, model.parameters(), optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.000 Acc@10: 0.000 Acc@100: 0.000 AccSum: 0.000
[epoch 1][dev ] Acc@1: 0.202 Acc@10: 0.433 Acc@100: 0.626 AccSum: 1.261
[epoch 1][test] Acc@1: 0.210 Acc@10: 0.446 Acc@100: 0.639 AccSum: 1.294
[best epoch: 1]
[epoch 2][dev ] Acc@1: 0.198 Acc@10: 0.420 Acc@100: 0.608 AccSum: 1.226
[epoch 3][dev ] Acc@1: 0.273 Acc@10: 0.526 Acc@100: 0.715 AccSum: 1.514
[epoch 3][test] Acc@1: 0.286 Acc@10: 0.542 Acc@100: 0.723 AccSum: 1.551
[best epoch: 3]
[epoch 4][dev ] Acc@1: 0.271 Acc@10: 0.520 Acc@100: 0.694 AccSum: 1.485
[epoch 5][dev ] Acc@1: 0.290 Acc@10: 0.562 Acc@100: 0.744 AccSum: 1.596
[epoch 5][test] Acc@1: 0.307 Acc@10: 0.575 Acc@100: 0.745 AccSum: 1.627
[best epoch: 5]
[epoch 6][dev ] Acc@1: 0.293 Acc@10: 0.543 Acc@100: 0.719 AccSum: 1.555
[epoch 7][dev ] Acc@1: 0.296 Acc@10: 0.545 Acc@100: 0.716 AccSum: 1.557
[epoch 8][dev ] Acc@1: 0.312 Acc@10: 0.569 Acc@100: 0.754 AccSum: 1.634
[epoch 8][test] Acc@1: 0.329 Acc@10: 0.588 Acc@100: 0.759 AccSum: 1.676
[best epoch: 8]


In [8]:
save_path = "bert_context_term_to_surface_attn_state_dict.pkl"
torch.save(best_sd, save_path)
model = multilevel_attention(input_size=768, target_size=768, lin=True).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

multilevel_attention(
  (self_attn): SelfAttention()
  (fc): Linear(in_features=768, out_features=768, bias=True)
)

In [9]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128, \
              dumb=True, GPU_INDEX=GPU_INDEX)
print ("MRR: %.3f" % mrr)
# gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=128, \
#             GPU_INDEX=GPU_INDEX)
# print ("MGD: %.3f MGD@10: %.3f" % gds)


MRR: 0.480


##### linear + relu

In [26]:
# input: contextual_term
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_surface_bert_ts100k_embeddings_all_names_mean_full_new.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"chv_term_embeddings_BERTbr_ts100k.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path,
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path, 
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [28]:
model = fc_aligner(input_size=768, target_size=768).cuda(GPU_INDEX)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = TripletLoss(margin=0.2, max_violation=True, device=GPU_INDEX)

best_sd = train(model, model.parameters(), optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.000 Acc@10: 0.000 Acc@100: 0.000 AccSum: 0.000
[epoch 1][dev ] Acc@1: 0.131 Acc@10: 0.341 Acc@100: 0.556 AccSum: 1.028
[epoch 1][test] Acc@1: 0.134 Acc@10: 0.353 Acc@100: 0.560 AccSum: 1.047
[best epoch: 1]
[epoch 2][dev ] Acc@1: 0.170 Acc@10: 0.406 Acc@100: 0.614 AccSum: 1.190
[epoch 2][test] Acc@1: 0.179 Acc@10: 0.420 Acc@100: 0.624 AccSum: 1.223
[best epoch: 2]
[epoch 3][dev ] Acc@1: 0.199 Acc@10: 0.440 Acc@100: 0.650 AccSum: 1.289
[epoch 3][test] Acc@1: 0.218 Acc@10: 0.457 Acc@100: 0.652 AccSum: 1.328
[best epoch: 3]
[epoch 4][dev ] Acc@1: 0.208 Acc@10: 0.453 Acc@100: 0.652 AccSum: 1.312
[epoch 4][test] Acc@1: 0.226 Acc@10: 0.475 Acc@100: 0.657 AccSum: 1.358
[best epoch: 4]
[epoch 5][dev ] Acc@1: 0.222 Acc@10: 0.466 Acc@100: 0.664 AccSum: 1.352
[epoch 5][test] Acc@1: 0.226 Acc@10: 0.490 Acc@100: 0.669 AccSum: 1.385
[best epoch: 5]
[epoch 6][dev ] Acc@1: 0.223 Acc@10: 0.482 Acc@100: 0.676 AccSum: 1.381
[epoch 6][test] Acc@1: 0.242 Acc@10: 0.497 Acc@100: 0.68

In [27]:
save_path = "bert_context_term_to_surface_state_dict.pkl"
#torch.save(best_sd, save_path)
model = fc_aligner(input_size=768, target_size=768).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

val_or_test(model, test_loader, testset.search_space_embeddings, \
                epoch=-1, typ="test", bsz=128, GPU_INDEX=GPU_INDEX)

[epoch 0][test] Acc@1: 0.320 Acc@10: 0.579 Acc@100: 0.748 AccSum: 1.647


[0.32045977011494253, 0.5788505747126437, 0.7480459770114942]

In [6]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128, \
              dumb=True, GPU_INDEX=GPU_INDEX)
print ("MRR: %.3f" % mrr)

gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=128, \
            GPU_INDEX=GPU_INDEX)
print ("MGD: %.3f MGD@10: %.3f" % gds)


MRR: 0.409



MGD: 0.496 MGD@10: 0.295


### fasttext align

#### term -> surface

In [6]:
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_surface_fasttext_embeddings_full.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"fasttext_term_embeddings.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path,
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path, 
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [8]:
model = fc_aligner(input_size=300, target_size=300).cuda(GPU_INDEX)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = TripletLoss(margin=0.2,max_violation=True,device=GPU_INDEX)

best_sd = train(model, model.parameters(), optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.000 Acc@10: 0.000 Acc@100: 0.000 AccSum: 0.000
[epoch 1][dev ] Acc@1: 0.061 Acc@10: 0.228 Acc@100: 0.490 AccSum: 0.779
[epoch 1][test] Acc@1: 0.057 Acc@10: 0.218 Acc@100: 0.484 AccSum: 0.759
[best epoch: 1]
[epoch 2][dev ] Acc@1: 0.136 Acc@10: 0.414 Acc@100: 0.693 AccSum: 1.242
[epoch 2][test] Acc@1: 0.128 Acc@10: 0.397 Acc@100: 0.686 AccSum: 1.211
[best epoch: 2]
[epoch 3][dev ] Acc@1: 0.180 Acc@10: 0.489 Acc@100: 0.749 AccSum: 1.417
[epoch 3][test] Acc@1: 0.177 Acc@10: 0.483 Acc@100: 0.744 AccSum: 1.404
[best epoch: 3]
[epoch 4][dev ] Acc@1: 0.208 Acc@10: 0.534 Acc@100: 0.783 AccSum: 1.524
[epoch 4][test] Acc@1: 0.208 Acc@10: 0.535 Acc@100: 0.780 AccSum: 1.523
[best epoch: 4]
[epoch 5][dev ] Acc@1: 0.234 Acc@10: 0.561 Acc@100: 0.801 AccSum: 1.596
[epoch 5][test] Acc@1: 0.236 Acc@10: 0.566 Acc@100: 0.798 AccSum: 1.600
[best epoch: 5]
[epoch 6][dev ] Acc@1: 0.250 Acc@10: 0.580 Acc@100: 0.813 AccSum: 1.642
[epoch 6][test] Acc@1: 0.256 Acc@10: 0.583 Acc@100: 0.81

In [10]:
save_path = "ft_term_to_surface_state_dict.pkl"
torch.save(best_sd, save_path)
model = fc_aligner(input_size=300, target_size=300).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

fc_aligner(
  (fc1): Linear(in_features=300, out_features=300, bias=True)
)

In [12]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128, GPU_INDEX=GPU_INDEX)
print ("MRR: %.3f" % mrr)

gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=128, GPU_INDEX=GPU_INDEX)
print ("MGRD: %.3f MGRD@10: %.3f" % gds)


MRR: 0.500



MGRD: 0.559 MGRD@10: 0.314


Process ForkPoolWorker-1503:
Process ForkPoolWorker-1507:
Process ForkPoolWorker-1493:
Process ForkPoolWorker-1502:
Process ForkPoolWorker-1513:
Process ForkPoolWorker-1509:
Process ForkPoolWorker-1492:
Process ForkPoolWorker-1496:
Process ForkPoolWorker-1501:
Process ForkPoolWorker-1505:
Process ForkPoolWorker-1506:
Process ForkPoolWorker-1510:
Process ForkPoolWorker-1512:
Process ForkPoolWorker-1495:
Process ForkPoolWorker-1508:
Process ForkPoolWorker-1511:
Process ForkPoolWorker-1500:
Process ForkPoolWorker-1494:
Process ForkPoolWorker-1491:
Process ForkPoolWorker-1499:
Traceback (most recent call last):
Process ForkPoolWorker-1497:
Traceback (most recent call last):
Process ForkPoolWorker-1504:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1498:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most re

#### term -> graph

In [13]:
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_node2vec_300d_20wl_embeddings.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"fasttext_term_embeddings.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path,
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path, 
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [15]:
model = fc_aligner(input_size=300, target_size=300).cuda(GPU_INDEX)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = TripletLoss(margin=0.2, max_violation=True, device=GPU_INDEX)

best_sd = train(model, model.parameters(), optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.0 Acc@10: 0.0 Acc@100: 0.1 AccSum: 0.2
[epoch 1][dev ] Acc@1: 0.4 Acc@10: 2.6 Acc@100: 11.1 AccSum: 14.2
[epoch 1][test] Acc@1: 0.5 Acc@10: 2.8 Acc@100: 11.7 AccSum: 14.9
[best epoch: 1]
[epoch 2][dev ] Acc@1: 1.5 Acc@10: 6.4 Acc@100: 19.9 AccSum: 27.8
[epoch 2][test] Acc@1: 1.5 Acc@10: 6.9 Acc@100: 20.7 AccSum: 29.1
[best epoch: 2]
[epoch 3][dev ] Acc@1: 2.0 Acc@10: 7.6 Acc@100: 23.8 AccSum: 33.4
[epoch 3][test] Acc@1: 2.2 Acc@10: 8.4 Acc@100: 24.5 AccSum: 35.1
[best epoch: 3]
[epoch 4][dev ] Acc@1: 2.6 Acc@10: 9.2 Acc@100: 27.2 AccSum: 39.0
[epoch 4][test] Acc@1: 2.7 Acc@10: 10.4 Acc@100: 28.1 AccSum: 41.2
[best epoch: 4]
[epoch 5][dev ] Acc@1: 3.4 Acc@10: 11.6 Acc@100: 31.0 AccSum: 46.0
[epoch 5][test] Acc@1: 3.8 Acc@10: 12.6 Acc@100: 31.7 AccSum: 48.0
[best epoch: 5]
[epoch 6][dev ] Acc@1: 3.6 Acc@10: 12.3 Acc@100: 32.4 AccSum: 48.3
[epoch 6][test] Acc@1: 4.0 Acc@10: 13.5 Acc@100: 33.0 AccSum: 50.5
[best epoch: 6]
[epoch 7][dev ] Acc@1: 3.8 Acc@10: 13.5 Acc

In [5]:
save_path = "term_to_graph_state_dict.pkl"
#torch.save(best_sd, save_path)
model = aligner(input_size=300, target_size=300).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

aligner(
  (fc1): Linear(in_features=300, out_features=300, bias=True)
)

In [6]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=256, \
              sample=False, dumb=True, GPU_INDEX=GPU_INDEX)
print ("MRR: %.3f" % mrr)

gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=256, sample=False,\
            GPU_INDEX=GPU_INDEX)
print ("MGD: %.3f MGD@10: %.3f" % gds)


MRR: 0.236



MGD: 0.403 MGD@10: 0.301


#### term -> surface+graph

In [14]:
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_surface_fasttext_embeddings_full+snomed_node2vec_300d_20wl_embeddings.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"fasttext_term_embeddings.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path,
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path, 
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [9]:
model = fc_aligner(input_size=300, target_size=600).cuda(GPU_INDEX)
train_params = model.parameters()
optimizer = torch.optim.AdamW(train_params, lr=1e-4)
criterion = TripletLoss(margin=0.2,max_violation=True, device=GPU_INDEX)

best_sd = train(model, train_params, optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.0 Acc@10: 0.0 Acc@100: 0.1 AccSum: 0.1
[epoch 1][dev ] Acc@1: 5.8 Acc@10: 20.4 Acc@100: 47.0 AccSum: 73.2
[epoch 1][test] Acc@1: 5.8 Acc@10: 20.3 Acc@100: 46.6 AccSum: 72.7
[best epoch: 1]
[epoch 2][dev ] Acc@1: 12.0 Acc@10: 36.1 Acc@100: 65.5 AccSum: 113.6
[epoch 2][test] Acc@1: 12.7 Acc@10: 37.0 Acc@100: 65.5 AccSum: 115.1
[best epoch: 2]
[epoch 3][dev ] Acc@1: 15.7 Acc@10: 42.7 Acc@100: 70.0 AccSum: 128.5
[epoch 3][test] Acc@1: 16.2 Acc@10: 42.9 Acc@100: 71.1 AccSum: 130.2
[best epoch: 3]
[epoch 4][dev ] Acc@1: 19.0 Acc@10: 46.8 Acc@100: 74.3 AccSum: 140.1
[epoch 4][test] Acc@1: 19.9 Acc@10: 48.0 Acc@100: 74.9 AccSum: 142.9
[best epoch: 4]
[epoch 5][dev ] Acc@1: 22.2 Acc@10: 51.6 Acc@100: 78.2 AccSum: 152.0
[epoch 5][test] Acc@1: 23.2 Acc@10: 52.8 Acc@100: 78.6 AccSum: 154.6
[best epoch: 5]
[epoch 6][dev ] Acc@1: 23.1 Acc@10: 53.5 Acc@100: 79.0 AccSum: 155.7
[epoch 6][test] Acc@1: 23.7 Acc@10: 54.5 Acc@100: 79.4 AccSum: 157.6
[best epoch: 6]
[epoch 7][dev ] 

In [5]:
save_path = "ft_term_to_surface+graph_state_dict.pkl"
#torch.save(best_sd, save_path)
model = fc_aligner(input_size=300, target_size=600).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

aligner(
  (fc1): Linear(in_features=300, out_features=600, bias=True)
)

In [6]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128, \
              sample=False, dumb=True, GPU_INDEX=GPU_INDEX)
print ("MRR: %.3f" % mrr)

gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=128, sample=False,\
            GPU_INDEX=GPU_INDEX)
print ("MGD: %.3f MGD@10: %.3f" % gds)


MRR: 0.571



MGD: 0.634 MGD@10: 0.358


### combine BERT with fasttext

#### ft_term+bert_term -> ft_surface+bert_surface+graph

In [15]:
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_surface_fasttext_embeddings_full+snomed_surface_bert_ts100k_embeddings_all_names_mean_full_new+snomed_node2vec_300d_20wl_embeddings.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"fasttext_term_embeddings+chv_term_embeddings_BERTbr_ts100k.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path,
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path, 
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [20]:
model = aligner(input_size=1068, target_size=1368).cuda(GPU_INDEX)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = TripletLoss(margin=0.2, max_violation=True, device=GPU_INDEX)

best_sd = train(model, model.parameters(), optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.0 Acc@10: 0.2 Acc@100: 0.3 AccSum: 0.5
[epoch 1][dev ] Acc@1: 18.5 Acc@10: 47.4 Acc@100: 70.6 AccSum: 136.5
[epoch 1][test] Acc@1: 20.7 Acc@10: 48.0 Acc@100: 70.7 AccSum: 139.5
[best epoch: 1]
[epoch 2][dev ] Acc@1: 25.8 Acc@10: 53.4 Acc@100: 75.8 AccSum: 155.0
[epoch 2][test] Acc@1: 27.1 Acc@10: 54.4 Acc@100: 76.3 AccSum: 157.8
[best epoch: 2]
[epoch 3][dev ] Acc@1: 30.1 Acc@10: 58.7 Acc@100: 77.9 AccSum: 166.8
[epoch 3][test] Acc@1: 31.1 Acc@10: 59.8 Acc@100: 79.2 AccSum: 170.1
[best epoch: 3]
[epoch 4][dev ] Acc@1: 31.6 Acc@10: 60.4 Acc@100: 79.9 AccSum: 171.9
[epoch 4][test] Acc@1: 33.1 Acc@10: 61.4 Acc@100: 80.8 AccSum: 175.4
[best epoch: 4]
[epoch 5][dev ] Acc@1: 33.8 Acc@10: 62.5 Acc@100: 81.6 AccSum: 177.8
[epoch 5][test] Acc@1: 34.9 Acc@10: 63.2 Acc@100: 82.5 AccSum: 180.7
[best epoch: 5]
[epoch 6][dev ] Acc@1: 35.4 Acc@10: 64.2 Acc@100: 83.5 AccSum: 183.0
[epoch 6][test] Acc@1: 37.0 Acc@10: 65.6 Acc@100: 84.2 AccSum: 186.8
[best epoch: 6]
[epoch 7][de

In [5]:
save_path = "ft_bert_context_term_to_ft_bert_surface+graph_state_dict.pkl"
#torch.save(best_sd, save_path)
model = fc_aligner(input_size=1068, target_size=1368).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

aligner(
  (fc1): Linear(in_features=1068, out_features=1368, bias=True)
)

In [6]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128, \
              GPU_INDEX=GPU_INDEX)
print ("MRR: %.3f" % mrr)

gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=128, \
             GPU_INDEX=GPU_INDEX)
print ("MGRD: %.3f MGRD@10: %.3f" % gds)


MRR: 0.622



MGRD: 0.667 MGRD@10: 0.351


In [6]:
# test back-off
accs, mrr, gd = compute_metrics_backoff(test_loader, testset, model, sf2id=None, train_dict=train_dict, \
                           topks=(1,10,100), bsz=128, GPU_INDEX=GPU_INDEX)
print ("[dict+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f MRR: %.3f MRGD@1: %.3f MRGD@10: %.3f" % \
       (accs[0], accs[1], accs[2], sum(accs), mrr, gd[0], gd[1]))

# test back-off
accs, mrr, gd = compute_metrics_backoff(test_loader, testset, model, sf2id=SF2ID, train_dict=None, \
                           topks=(1,10,100), bsz=128, GPU_INDEX=GPU_INDEX)
print ("[smatch+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f MRR: %.3f MRGD@1: %.3f MRGD@10: %.3f" % \
       (accs[0], accs[1], accs[2], sum(accs), mrr, gd[0], gd[1]))

# test back-off
accs, mrr, gd = compute_metrics_backoff(test_loader, testset, model, sf2id=SF2ID, train_dict=train_dict, \
                           topks=(1,10,100), bsz=128, GPU_INDEX=GPU_INDEX)

print ("[dict+smatch+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f MRR: %.3f MRGD@1: %.3f MRGD@10: %.3f" % \
       (accs[0], accs[1], accs[2], sum(accs), mrr, gd[0], gd[1]))

ratio solved by heuristics: 50.7%
[dict+model] Acc@1: 0.727 Acc@10: 0.859 Acc@100: 0.927 Acc sum: 2.513 MRR: 0.774 MRGD@1: 0.806 MRGD@10: 0.674
ratio solved by heuristics: 40.1%
[smatch+model] Acc@1: 0.654 Acc@10: 0.829 Acc@100: 0.918 Acc sum: 2.402 MRR: 0.716 MRGD@1: 0.752 MRGD@10: 0.598
ratio solved by heuristics: 65.6%
[dict+smatch+model] Acc@1: 0.785 Acc@10: 0.876 Acc@100: 0.932 Acc sum: 2.593 MRR: 0.817 MRGD@1: 0.844 MRGD@10: 0.766


#### ft_term+bert_term -> ft_surface+bert_surface

In [16]:
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
snomed_vec_path = os.path.join(FEATURE_PATH, "snomed_surface_fasttext_embeddings_full+snomed_surface_bert_ts100k_embeddings_all_names_mean_full_new.pkl")
term_vec_path = os.path.join(FEATURE_PATH,"fasttext_term_embeddings+chv_term_embeddings_BERTbr_ts100k.pkl")

train_loader, _ = get_loader_single(FULL_CHV_PATH, chv_train_path,
                                    term_vec_path, snomed_vec_path, 
                                    batch_size=64, shuffle=True, 
                                    num_workers=10, gran="general")
val_loader, valset = get_loader_single(FULL_CHV_PATH, chv_dev_path, 
                                       term_vec_path, snomed_vec_path, 
                                       batch_size=64, shuffle=False, 
                                       num_workers=10, gran="general")
test_loader, testset = get_loader_single(FULL_CHV_PATH, chv_test_path, 
                                         term_vec_path, snomed_vec_path, 
                                         batch_size=64, shuffle=False, 
                                         num_workers=10, gran="general", 
                                         load_target=True)

[target embeddings loaded, search space size: 350830]


In [7]:
model = fc_aligner(input_size=1068, target_size=1068).cuda(GPU_INDEX)
train_params = model.parameters()
optimizer = torch.optim.AdamW(train_params, lr=1e-4)
criterion = TripletLoss(margin=0.2, max_violation=True, device=GPU_INDEX)

best_sd = train(model, train_params, optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=50, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.0 Acc@10: 0.0 Acc@100: 0.1 AccSum: 0.1
[epoch 1][dev ] Acc@1: 18.7 Acc@10: 44.5 Acc@100: 70.0 AccSum: 133.2
[epoch 1][test] Acc@1: 19.4 Acc@10: 44.9 Acc@100: 69.7 AccSum: 134.0
[best epoch: 1]
[epoch 2][dev ] Acc@1: 23.4 Acc@10: 50.2 Acc@100: 75.0 AccSum: 148.6
[epoch 2][test] Acc@1: 22.6 Acc@10: 52.1 Acc@100: 73.6 AccSum: 148.3
[best epoch: 2]
[epoch 3][dev ] Acc@1: 25.5 Acc@10: 51.7 Acc@100: 76.2 AccSum: 153.4
[epoch 3][test] Acc@1: 22.8 Acc@10: 52.2 Acc@100: 75.1 AccSum: 150.1
[best epoch: 3]
[epoch 4][dev ] Acc@1: 27.6 Acc@10: 52.1 Acc@100: 76.8 AccSum: 156.5
[epoch 4][test] Acc@1: 24.8 Acc@10: 53.7 Acc@100: 75.5 AccSum: 154.0
[best epoch: 4]
[epoch 5][dev ] Acc@1: 28.8 Acc@10: 53.6 Acc@100: 77.5 AccSum: 159.9
[epoch 5][test] Acc@1: 26.1 Acc@10: 55.2 Acc@100: 76.7 AccSum: 158.0
[best epoch: 5]
[epoch 6][dev ] Acc@1: 28.5 Acc@10: 53.6 Acc@100: 77.3 AccSum: 159.3
[epoch 7][dev ] Acc@1: 29.6 Acc@10: 52.3 Acc@100: 77.5 AccSum: 159.3
[epoch 8][dev ] Acc@1: 30.0 

In [8]:
save_path = "ft_bert_context_term_to_ft_bert_surface_state_dict.pkl"
torch.save(best_sd, save_path)
model = aligner(input_size=1068, target_size=1068).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

aligner(
  (fc1): Linear(in_features=1068, out_features=1068, bias=True)
)

In [10]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128, \
              sample=False, dumb=True, GPU_INDEX=GPU_INDEX)
print ("MRR: %.3f" % mrr)


MRR: 0.390


In [11]:
gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=256, sample=False, GPU_INDEX=GPU_INDEX)
print ("MGRD: %.3f MGRD@10: %.3f" % gds)


MGRD: 0.496 MGRD@10: 0.314


####  ft_term+bert_term (multilevel attention) -> ft_surface+bert_surface(+graph)

In [21]:
chv_train_path = os.path.join(DATA_DIR, 'train.csv')
chv_dev_path = os.path.join(DATA_DIR, 'dev.csv')
chv_test_path = os.path.join(DATA_DIR, 'test.csv')
#snomed_vec_path = os.path.join(FEATURE_PATH,"snomed_surface_fasttext_embeddings_full+snomed_surface_bert_ts100k_embeddings_all_names_mean_full_last_layer_with_ST.pkl")
snomed_vec_path = os.path.join(FEATURE_PATH,"snomed_surface_fasttext_embeddings_full+snomed_surface_bert_ts100k_embeddings_all_names_mean_full_last_layer_with_ST+snomed_node2vec_300d_20wl_embeddings.pkl")
term_vec_path1 = os.path.join(FEATURE_PATH,"fasttext_term_embeddings.pkl")
term_vec_path2 = os.path.join(FEATURE_PATH,"chv_term_embeddings_BERTbr_ts100k_multilevel_all.pkl")

gran="general"

train_dict = build_train_dict(chv_train_path, gran=gran)

train_loader, _ = get_loader_mla(FULL_CHV_PATH, chv_train_path, 
                                 term_vec_path1, term_vec_path2, 
                                 snomed_vec_path, batch_size=64, 
                                 shuffle=True, num_workers=10, gran=gran)
val_loader, valset = get_loader_mla(FULL_CHV_PATH, chv_dev_path, 
                                    term_vec_path1, term_vec_path2, 
                                    snomed_vec_path, batch_size=64, 
                                    shuffle=False, num_workers=10, gran=gran)
test_loader, testset = get_loader_mla(FULL_CHV_PATH, chv_test_path, 
                                      term_vec_path1, term_vec_path2, 
                                      snomed_vec_path, batch_size=64,
                                      shuffle=False,  num_workers=10, 
                                      gran=gran, load_target=True)

[target embeddings loaded, search space size: 350830]


In [22]:
model = mla_bert_ft_ensemble(target_len=1368).cuda(GPU_INDEX)
#save_path = "bert_context_term_to_surface_attn_state_dict.pkl"
#model.mla_bert.load_state_dict(torch.load(save_path))

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = TripletLoss(margin=0.2, max_violation=True, device=GPU_INDEX)

best_sd = train_joint(model, model.parameters(), optimizer, criterion, \
      train_loader, val_loader, valset, test_loader, \
      testset, num_epoch=100, dor=0.0, GPU_INDEX=GPU_INDEX)

[epoch 0][dev ] Acc@1: 0.000 Acc@10: 0.000 Acc@100: 0.000 AccSum: 0.000
[epoch 1][test] Acc@1: 0.034 Acc@10: 0.135 Acc@100: 0.322 AccSum: 0.491
[best epoch: 1]
[epoch 2][test] Acc@1: 0.182 Acc@10: 0.439 Acc@100: 0.646 AccSum: 1.266
[best epoch: 2]
[epoch 3][test] Acc@1: 0.303 Acc@10: 0.564 Acc@100: 0.756 AccSum: 1.623
[best epoch: 3]
[epoch 4][test] Acc@1: 0.334 Acc@10: 0.591 Acc@100: 0.781 AccSum: 1.707
[best epoch: 4]
[epoch 5][test] Acc@1: 0.349 Acc@10: 0.610 Acc@100: 0.794 AccSum: 1.754
[best epoch: 5]
[epoch 6][test] Acc@1: 0.377 Acc@10: 0.638 Acc@100: 0.817 AccSum: 1.832
[best epoch: 6]
[epoch 7][test] Acc@1: 0.386 Acc@10: 0.650 Acc@100: 0.821 AccSum: 1.857
[best epoch: 7]
[epoch 8][test] Acc@1: 0.404 Acc@10: 0.676 Acc@100: 0.844 AccSum: 1.924
[best epoch: 8]
[epoch 9][test] Acc@1: 0.419 Acc@10: 0.687 Acc@100: 0.852 AccSum: 1.959
[best epoch: 9]
[epoch 10][test] Acc@1: 0.429 Acc@10: 0.700 Acc@100: 0.862 AccSum: 1.991
[best epoch: 10]
[epoch 11][test] Acc@1: 0.436 Acc@10: 0.714 Ac

In [23]:
save_path = "ensemble_w_graph_state_dict.pkl"
#torch.save(best_sd, save_path)
model = mla_bert_ft_ensemble(target_len=1368).cuda(GPU_INDEX)
model.load_state_dict(torch.load(save_path))
model.eval()

mla_bert_ft_ensemble(
  (mla_bert): multilevel_attention(
    (self_attn): SelfAttention()
  )
  (fc): Linear(in_features=1068, out_features=1368, bias=True)
  (fc_aligner): fc_aligner(
    (fc1): Linear(in_features=300, out_features=300, bias=True)
  )
)

In [24]:
mrr = evalMRR(test_loader, model, testset.search_space_embeddings, bsz=128, \
              dumb=True, GPU_INDEX=GPU_INDEX, ensemble=True)
print ("MRR: %.3f" % mrr)

gds = evalGD(test_loader, testset, model, topk=10, dor=0.0, bsz=128, \
             dumb=False, GPU_INDEX=GPU_INDEX, ensemble=True)
print ("MGRD: %.3f MGRD@10: %.3f" % gds)


MRR: 0.742



MGRD: 0.770 MGRD@10: 0.372


Process ForkPoolWorker-3558:
Process ForkPoolWorker-3565:
Process ForkPoolWorker-3567:
Process ForkPoolWorker-3561:
Process ForkPoolWorker-3560:
Process ForkPoolWorker-3555:
Process ForkPoolWorker-3576:
Process ForkPoolWorker-3574:
Process ForkPoolWorker-3557:
Process ForkPoolWorker-3554:
Process ForkPoolWorker-3572:
Process ForkPoolWorker-3563:
Process ForkPoolWorker-3556:
Process ForkPoolWorker-3573:
Process ForkPoolWorker-3564:
Process ForkPoolWorker-3575:
Process ForkPoolWorker-3559:
Process ForkPoolWorker-3566:
Process ForkPoolWorker-3562:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call l

In [25]:
# Dict + Neural
# accs, pred_dict = compute_metrics_backoff(test_loader, testset, model, sf2id=None, train_dict=train_dict, \
#                            ed_dict=None, topks=(1,10,100), bsz=128, GPU_INDEX=GPU_INDEX, ensemble=True)
# print ("[dict+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f" % \
#        (accs[0], accs[1], accs[2], sum(accs)))

# # Exact Match + Neural
# accs, pred_dict= compute_metrics_backoff(test_loader, testset, model, sf2id=SF2ID, train_dict=None, \
#                            ed_dict=None, topks=(1,10,100), bsz=128, GPU_INDEX=GPU_INDEX, ensemble=True)
# print ("[smatch+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f" % \
#        (accs[0], accs[1], accs[2], sum(accs)))

# # Dict + Exact Match + Neural
# accs, pred_dict= compute_metrics_backoff(test_loader, testset, model, sf2id=SF2ID, train_dict=train_dict, \
#                            ed_dict=None, topks=(1,10,100), bsz=128, GPU_INDEX=GPU_INDEX, ensemble=True)
# print ("[dict+smatch+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f" % \
#        (accs[0], accs[1], accs[2], sum(accs)))

# Dict + Stoilos + Neural
accs, pred_dict= compute_metrics_backoff(test_loader, testset, model, sf2id=SF2ID, train_dict=train_dict, \
                           ed_dict=None, topks=(1,10,100), bsz=128, GPU_INDEX=GPU_INDEX, ensemble=True)
print ("[dict+smatch+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f" % \
       (accs[0], accs[1], accs[2], sum(accs)))

#print ("[dict+smatch+model] Acc@1: %.3f Acc@10: %.3f Acc@100: %.3f Acc sum: %.3f MRR: %.3f MRGD@1: %.3f MRGD@10: %.3f" % \
#       (accs[0], accs[1], accs[2], sum(accs), mrr, gd[0], gd[1]))

ratio attempted by heuristics: 69.0%
0.6554022988505747
[dict+smatch+model] Acc@1: 0.792 Acc@10: 0.878 Acc@100: 0.920 Acc sum: 2.590
